整体的思路是，将API specification文档先输入给LLM，然后将用户的query输入给LLM，这样LLM就能产生一个能call的URL。  
然后通过调用这个URL，得到对应的respond。  
再将respond和query输入给LLM，产生最终的答案。

方法1：直接使用OpenAI自带的函数调用功能


In [ ]:
from langchain.chains.openai_functions.openapi import get_openapi_chain
import os
os.environ["OPENAI_API_KEY"] = ""

chain = get_openapi_chain(
    "https://api.speak.com/openapi.yaml"
)
chain("translate \'I love you \' into German")

方法2：使用自定义的Chain


In [12]:
from langchain.chains import APIChain
from langchain.chains.api import open_meteo_docs
from langchain.llms import OpenAI
api_doc = r'BASE URL: https://api.open-meteo.com/\n\nAPI Documentation\nThe API endpoint /v1/forecast accepts a geographical coordinate, a list of weather variables and responds with a JSON hourly weather forecast for 7 days. Time always starts at 0:00 today and contains 168 hours. All URL parameters are listed below:\n\nParameter\tFormat\tRequired\tDefault\tDescription\nlatitude, longitude\tFloating point\tYes\t\tGeographical WGS84 coordinate of the location\nhourly\tString array\tNo\t\tA list of weather variables which should be returned. Values can be comma separated, or multiple &hourly= parameter in the URL can be used.\ndaily\tString array\tNo\t\tA list of daily weather variable aggregations which should be returned. Values can be comma separated, or multiple &daily= parameter in the URL can be used. If daily weather variables are specified, parameter timezone is required.\ncurrent_weather\tBool\tNo\tfalse\tInclude current weather conditions in the JSON output.\ntemperature_unit\tString\tNo\tcelsius\tIf fahrenheit is set, all temperature values are converted to Fahrenheit.\nwindspeed_unit\tString\tNo\tkmh\tOther wind speed speed units: ms, mph and kn\nprecipitation_unit\tString\tNo\tmm\tOther precipitation amount units: inch\ntimeformat\tString\tNo\tiso8601\tIf format unixtime is selected, all time values are returned in UNIX epoch time in seconds. Please note that all timestamp are in GMT+0! For daily values with unix timestamps, please apply utc_offset_seconds again to get the correct date.\ntimezone\tString\tNo\tGMT\tIf timezone is set, all timestamps are returned as local-time and data is returned starting at 00:00 local-time. Any time zone name from the time zone database is supported. If auto is set as a time zone, the coordinates will be automatically resolved to the local time zone.\npast_days\tInteger (0-2)\tNo\t0\tIf past_days is set, yesterday or the day before yesterday data are also returned.\nstart_date\nend_date\tString (yyyy-mm-dd)\tNo\t\tThe time interval to get weather data. A day must be specified as an ISO8601 date (e.g. 2022-06-30).\nmodels\tString array\tNo\tauto\tManually select one or more weather models. Per default, the best suitable weather models will be combined.\n\nHourly Parameter Definition\nThe parameter &hourly= accepts the following values. Most weather variables are given as an instantaneous value for the indicated hour. Some variables like precipitation are calculated from the preceding hour as an average or sum.\n\nVariable\tValid time\tUnit\tDescription\ntemperature_2m\tInstant\t°C (°F)\tAir temperature at 2 meters above ground\nsnowfall\tPreceding hour sum\tcm (inch)\tSnowfall amount of the preceding hour in centimeters. For the water equivalent in millimeter, divide by 7. E.g. 7 cm snow = 10 mm precipitation water equivalent\nrain\tPreceding hour sum\tmm (inch)\tRain from large scale weather systems of the preceding hour in millimeter\nshowers\tPreceding hour sum\tmm (inch)\tShowers from convective precipitation in millimeters from the preceding hour\nweathercode\tInstant\tWMO code\tWeather condition as a numeric code. Follow WMO weather interpretation codes. See table below for details.\nsnow_depth\tInstant\tmeters\tSnow depth on the ground\nfreezinglevel_height\tInstant\tmeters\tAltitude above sea level of the 0°C level\nvisibility\tInstant\tmeters\tViewing distance in meters. Influenced by low clouds, humidity and aerosols. Maximum visibility is approximately 24 km.'

llm = OpenAI(temperature=0)
chain = APIChain.from_llm_and_api_docs(
    llm,
    api_doc,
    verbose=True,
    limit_to_domains=["https://api.open-meteo.com/"],
)
chain.run(
    "What is the weather like right now in Munich, Germany in degrees Fahrenheit?"
)



> Entering new APIChain chain...
https://api.open-meteo.com/v1/forecast?latitude=48.1351&longitude=11.5820&hourly=temperature_2m&temperature_unit=fahrenheit&current_weather=true
{"latitude":48.14,"longitude":11.58,"generationtime_ms":0.0680685043334961,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":521.0,"current_weather_units":{"time":"iso8601","interval":"seconds","temperature":"°F","windspeed":"km/h","winddirection":"°","is_day":"","weathercode":"wmo code"},"current_weather":{"time":"2023-12-14T10:30","interval":900,"temperature":41.7,"windspeed":9.0,"winddirection":265,"is_day":1,"weathercode":80},"hourly_units":{"time":"iso8601","temperature_2m":"°F"},"hourly":{"time":["2023-12-14T00:00","2023-12-14T01:00","2023-12-14T02:00","2023-12-14T03:00","2023-12-14T04:00","2023-12-14T05:00","2023-12-14T06:00","2023-12-14T07:00","2023-12-14T08:00","2023-12-14T09:00","2023-12-14T10:00","2023-12-14T11:00","2023-12-14T12:00","2023-12-14T13:00","2023-12-14T14

' The current temperature in Munich, Germany is 41.7°F.'

这里遇到一个很实际的问题，那就是在使用API的时候，很容易遇到API spec 无法解析，或者产生的URL无效等问题。这些往往和parser 的版本，API Doc的版本相关。其实本身并不鲁棒。这就是使用URL或者说GET的不方便的地方。
那么能不能让这个过程变得更加简单，变得更加鲁棒呢？

In [6]:
open_meteo_docs.OPEN_METEO_DOCS # 就是一个字符串，属于是api specification 

'BASE URL: https://api.open-meteo.com/\n\nAPI Documentation\nThe API endpoint /v1/forecast accepts a geographical coordinate, a list of weather variables and responds with a JSON hourly weather forecast for 7 days. Time always starts at 0:00 today and contains 168 hours. All URL parameters are listed below:\n\nParameter\tFormat\tRequired\tDefault\tDescription\nlatitude, longitude\tFloating point\tYes\t\tGeographical WGS84 coordinate of the location\nhourly\tString array\tNo\t\tA list of weather variables which should be returned. Values can be comma separated, or multiple &hourly= parameter in the URL can be used.\ndaily\tString array\tNo\t\tA list of daily weather variable aggregations which should be returned. Values can be comma separated, or multiple &daily= parameter in the URL can be used. If daily weather variables are specified, parameter timezone is required.\ncurrent_weather\tBool\tNo\tfalse\tInclude current weather conditions in the JSON output.\ntemperature_unit\tString\tNo

In [13]:
import os

os.environ["TMDB_BEARER_TOKEN"] = "" # 需要去申请
from langchain.chains.api import tmdb_docs
# 但是这里可以看到的是，如果你需要验证登录，那么就需要添加 header
headers = {"Authorization": f"Bearer {os.environ['TMDB_BEARER_TOKEN']}"}
chain = APIChain.from_llm_and_api_docs(
    llm,
    tmdb_docs.TMDB_DOCS,
    headers=headers,
    verbose=True,
    limit_to_domains=["https://api.themoviedb.org/"],
)
chain.run("Search for 'Avatar'")



> Entering new APIChain chain...
 https://api.themoviedb.org/3/search/movie?query=Avatar&language=en-US
{"status_code":7,"status_message":"Invalid API key: You must be granted a valid key.","success":false}


> Finished chain.


' The API call was unsuccessful due to an invalid API key.'

方法3：使用GET方法

In [14]:
from langchain.chains import LLMChain, LLMRequestsChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
template = """Between >>> and <<< are the raw search result text from google.
Extract the answer to the question '{query}' or say "not found" if the information is not contained.
Use the format
Extracted:<answer or "not found">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)

In [15]:
# the LLMRequestsChain makes an HTTP GET request.
chain = LLMRequestsChain(llm_chain=LLMChain(llm=OpenAI(temperature=0), prompt=PROMPT))
question = "What are the Three (3) biggest countries, and their respective sizes?"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+"),
}
chain(inputs)
# 这里使用了Google search的例子。那么同样，你应该也能用这个接口来搜索其他内容。也就是能联网搜索了。原来联网搜索这么简单。

{'query': 'What are the Three (3) biggest countries, and their respective sizes?',
 'url': 'https://www.google.com/search?q=What+are+the+Three+(3)+biggest+countries,+and+their+respective+sizes?',
 'output': ' Russia (17.1M km²), Canada (10M km²), China (9.7M km²)'}